# Применение модели

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json

In [2]:
# Вспомогательные функции
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [4]:
# Загрузим словарь
with open('vocab.json', 'r') as f:
  vocab = json.load(f)

# Загрузим типы задач
with open('tag_to_ix.json', 'r') as f:
  tag_to_ix = json.load(f)

In [5]:
# Model:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [6]:
# Загрузим модель
model = torch.load("LSTM_NER.pth")
model.eval()

LSTMTagger(
  (word_embeddings): Embedding(1239, 6)
  (lstm): LSTM(6, 6)
  (hidden2tag): Linear(in_features=6, out_features=4, bias=True)
)

In [16]:
def test_result(test_case):
  test_case = test_case.split()
  with torch.no_grad():
      inputs = prepare_sequence(test_case, vocab)
      tag_scores = model(inputs)
      results = [(tag_scores == max(i)).nonzero(as_tuple=False)[0][1].item() for i in tag_scores]
      ix_to_tag = {v:k for k,v in tag_to_ix.items()}
      translations = [ix_to_tag[i] for i in results]
  print(' '.join(test_case))
  print("TASK: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TASK"])))
  print("PERSON: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="PERSON"])))
  print("TIME: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TIME"])))
  print('')

In [17]:
examples = ["Создай задачу : Сергей должен предоставить 3 варианта дизайна упаковки до конца этой недели",
"Создай задачу : Петр требуется верстка лендинга в течение 2 дней после утверждения макета",
"Создай задачу : Олег нужны эскизы интерьера офиса к понедельнику",
"Создай задачу : Анна пожалуйста разработай несколько вариантов баннера ко Дню рождения компании ко 2 апреля",
"Создай задачу : Попросить Никиту разработать фирменный стиль для ресторана к 1 апреля"]

for test_case in examples:
  test_result(test_case)



Создай задачу : Сергей должен предоставить 3 варианта дизайна упаковки до конца этой недели
TASK: Создай задачу
PERSON: Сергей
TIME: до конца этой недели

Создай задачу : Петр требуется верстка лендинга в течение 2 дней после утверждения макета
TASK: Создай задачу
PERSON: после
TIME: 2 дней

Создай задачу : Олег нужны эскизы интерьера офиса к понедельнику
TASK: Создай задачу
PERSON: Олег
TIME: 

Создай задачу : Анна пожалуйста разработай несколько вариантов баннера ко Дню рождения компании ко 2 апреля
TASK: Создай задачу
PERSON: Анна
TIME: 

Создай задачу : Попросить Никиту разработать фирменный стиль для ресторана к 1 апреля
TASK: Создай задачу
PERSON: Никиту
TIME: 1 апреля



In [18]:
# Не работает со словами, которые не участвовали в обучении
test_result("Создай задачу : Олеся должна предоставить эскиз нового карнавального костюма завтра к 11 : 00")

KeyError: ignored